In [6]:
import os
import numpy as np

In [11]:
data_str = 'imagenet'
main_model_name = new_model_name = 'dinov2_vits14'
main_lr = new_lr = 0.000001
original_num_epochs = 8
binary_results_path = '../binary_results'
binary_files = [f for f in os.listdir(binary_results_path) if f.startswith('imagenet_dinov2_vits14')]
file_0 = np.load(fr'{binary_results_path}/{binary_files[0]}')
file_0.shape

(2100,)